# Exploração e limpeza de dados de contas de crédito de titulares inadimplentes após seis meses 

**Assunto**: Exploração e limpeza de dados

**Data**: 16/05/2023

**Contexto**: Foi recebido de uma empresa de cartão de crédito, um dataset que inclui dados domográficos e dados financeiros recentes (últimos seis meses) de uma amostra de 30.000 titulares de contas. A **definição de linha** desse dataset é contas de crédito. O dados trazem o titular que deixou de realizar o pagamento mínimo no mês seguinte ao período histórico de seis meses.

**Objetivo**: Desenvolver um modelo que preveja se uma conta ficará inadimplente no próximo mês, de acordo com dados demográficos e históricos.

## Vrerificação da integridade dos dados

### Importação do dados a partir de um arquivo `xls` (exel)

In [3]:
# Importação do pacote Pandas
import pandas as pd

O método `pd.read_excel` carrega o arquivo que está em formato excel no diretório dos dados:

In [4]:
df = pd.read_excel('../data/default_of_credit_card_clients__courseware_version_1_21_19.xls')

Isso gera um **DataFrame** que é chamado de `df`.

1. `df.shape` para examinar o número de linas e colunas: 

In [5]:
df.shape

(30000, 25)

2. `df.info` para obter informações sobre todas as colunas: 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

Usando o método `df.colums` podemos examinar os nomes de todas as colunas:

In [8]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

**Dicionario dos dados**:

É fundamental, quando recebemos um dataset que não conhecemos, solicitar também o dicion
Apesar de não ter encontrado o dicionário dos dados no repositório, como autor solicita, o mesmo faz um resumo breve do significam as colunas.

- **LIMIT_BAL**: Valor do cédito fornecido (em dólares taiwaneses (NT)) inclusive o cŕedito do consumidor individual e familiar(complementar.
- **SEX**: Gênero (1 = masculino; 2 = feminino).
- **EDUCATION**: Instrução (1 = pós-graduação; 2 = universidade; 3 = ensino médio; 4 = outros.
- **MARRIAGE**:Estado civil (1 = casado; 2 = solteiro; 3 = outros).
- **AGE**: Idade (ano).
- **PAY_1 - PAY_6**: Registro de pagamentos passados. Pagamentos mensais passados, registrados de abril a setembro.
- **PAY_1**: Representa o status de reembolso em Setembro; `PAY_2` = estatus de Agosto; `PAY_3` = status de Julho, sucessivamente, até Abril. 
    A escala de medida do status de reembolso é a seguinte: 1 = atraso de um mês no pagamento; 2 = atraso de dois meses no pagamento; assim por diante até 8 = atraso de oito meses no pagamento; 9 = atraso de nove meses ou mais no pagamento. 
- **BILL_ATM1 - BILL_ATM6**: Valor da fatura (em dólares taiwaneses)
- **BILL_ATM1**: Representa o valor da fatura em Setembro; `BILL_ATM2` representa a fatura de Agosto; assim por diante até `BILL_ATM7`, que representa o valor da fatura em Abril.
- **PAY_ATM1 - PAY_ATM6**: Valor de pagamentos anteriores (novos dólares taiwaneses).
- **PAY_ATM1**: Representa o valor pago em Setembro; `PAY_ATM2` representa a fatura de Agosto; assim por diante até `PAY_ATM6` que representa o valor pago em Abril.



### Verificando a integridade dos dados

Vimos com o método `df.shape` que o dataset possui 30000 registros (amostra) que equivalem há 30000 observações referentes a contas de crédito. E 25 colunas que equivalem às variáveis que podem ser características, resposta ou métadado.  

1. Verificando se o número de identificadores únicos corresponde à quantidade de registros no dataset: 
    A primeira coisa a se pensar é: "Esse dataset está correto?". Essa verificação pode ser feita nos sertificando que a quantidade de registros é igual a quantidade de identificadores únicos do dataset. Assim garantimos que a quantidade da amostra está correta. 
    
O método `head()` é usados para vermos como está o aspecto dos dados através da visualização dos primeiros registros:

In [7]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


> ver nota sobre daos tabulares e tipos de dados

Notamos que a coluna `ID` contém os valores exclusivos. Para verificar se isso acontece em toda a base de dados, usamos o método `nunique()` na **serie**, ou seja, na coluna que queremos observar:

In [12]:
df['ID'].nunique()

29687

A saida do método acima indica que temos uma diferença entre a quantidadde de linhas e a quantidade de identificadores únicos. Isso significa que possívelmente temos IDs duplicados, mas em que quantidade? 
Para começarmos a reponder a essa pergunta, podemos usar o método `value_counts()` na série `ID`. Esse método é semelhante ao procedimento `GROUP BY/COUNT` do SQL e listará os IDs exclusivos e **frequência** com que ocorrem. Para obter os valores, o resultado é armazenado na variável `id_counts` e exibido com o método `head()`:

In [16]:
id_counts = df['ID'].value_counts()
id_counts.head() 

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
Name: ID, dtype: int64

In [18]:
# Exibindo as entradas duplicadas agrupadas executando outra contagem de valores: 
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

A saída mostra que a maioria dos IDs apenas uma vez, como esperado, contudo, existem 313 IDs que ocorrem 2 vezes. Para examinarmos alguns desses IDs duplicados usaremo o conceito de <a href="./Entendendo_máscaras_booleanas.ipynb" >**máscaras booleanas**.</a> 